# Sistema de recomendação para filmes do IMDB

Inicialmente utilizamos a API do IMDB para pegar a base de filmes com 10.000 filmes. Partindo dessa base vamos montar um sistema de recomendação Content-Based.
Para este estudo primeiro iremos vetorizar nossos filmes e comparar eles através da similaridade de cossenos. O intuito deste notebook é aprofundar e explorar conceitos usados na criação de modelos de machine learning, principalmente em sistemas de recomendações.

Existem três tipos de sistemas de recomendações:

*  Content-based filtering: Esse sistema de recomendação leva em consideração itens curtidos anteriormente pelo usuário para fazer novas recomendações de produtos. Por exemplo se o usuário gostou de filmes de comédia, novos filmes desse gênero serão recomendados a ele. Esse filtro pode ser aplicado ao item e ao usuário.

* Collaborative filtering: Observa as semlhanças entre itens e usuários ao mesmo tempo. Ele leva em consideração similiridade de gostos de usuários e quais itens eles gostaram. Por exemplo: usuário A consome filmes semelhantes ao usuário B, logo, se o usuário A experimentar e gostar de um novo filme o sistema recomendará para o usuário B.

* Hybrid: É uma combinação entre os dois metódos citados anteriormente.

No nosso modelos adotaremos um sistema do primeiro tipo.

## Importando bibliotecas

In [11]:
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.decomposition import TruncatedSVD
from sklearn.metrics.pairwise import cosine_similarity

## Importando e verificando dados

In [4]:
def load_csv_data(path):
    df = pd.read_csv(path)
    return df

df_filmes = load_csv_data('dados/filmes.csv')
df_filmes.head()

,id,id_genero,titulo,resumo,lancamento,idioma_original
0,238,"[18, 80]",The Godfather,"Spanning the years 1945 to 1955, a chronicle o...",1972-03-14,en
1,278,"[18, 80]",The Shawshank Redemption,Framed in the 1940s for the double murder of h...,1994-09-23,en
2,240,"[18, 80]",The Godfather Part II,In the continuing saga of the Corleone crime f...,1974-12-20,en
3,424,"[18, 36, 10752]",Schindler's List,The true story of how businessman Oskar Schind...,1993-12-15,en
4,19404,"[35, 18, 10749]",Dilwale Dulhania Le Jayenge,"Raj is a rich, carefree, happy-go-lucky second...",1995-10-19,hi


In [5]:
df_generos = load_csv_data('dados/generos.csv')
df_generos.head()

,id,name
0,28,Action
1,12,Adventure
2,16,Animation
3,35,Comedy
4,80,Crime


In [4]:
df_filmes.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 6 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   id               10000 non-null  int64 
 1   id_genero        10000 non-null  object
 2   titulo           10000 non-null  object
 3   resumo           9998 non-null   object
 4   lancamento       10000 non-null  object
 5   idioma_original  10000 non-null  object
dtypes: int64(1), object(5)
memory usage: 468.9+ KB


In [6]:
df_generos.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 19 entries, 0 to 18
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   id      19 non-null     int64 
 1   name    19 non-null     object
dtypes: int64(1), object(1)
memory usage: 432.0+ bytes


Como só temos duas linhas com o resumo vazio escolhi dropar essas linhas já que não vão interferir na análise

In [5]:
df_filmes = df_filmes.dropna(axis=0, how='any')
len(df_filmes)

9998

## Criação e seleção de features

Unindo dados de filmes e generos:

In [7]:
# Função para obter os nomes dos gêneros
def obter_nomes_generos(ids):
    generos = df_generos[df_generos['id'].isin(ids)]
    nomes = generos['name'].tolist()
    return ', '.join(nomes)

# Aplicar a função para obter os nomes dos gêneros para cada filme
df_filmes['genero'] = df_filmes['id_genero'].apply(lambda x: obter_nomes_generos(eval(x)))
df_filmes['genero'] = df_filmes['genero'].str.replace(',', '')
df_filmes.head()

,id,id_genero,titulo,resumo,lancamento,idioma_original,genero
0,238,"[18, 80]",The Godfather,"Spanning the years 1945 to 1955, a chronicle o...",1972-03-14,en,Crime Drama
1,278,"[18, 80]",The Shawshank Redemption,Framed in the 1940s for the double murder of h...,1994-09-23,en,Crime Drama
2,240,"[18, 80]",The Godfather Part II,In the continuing saga of the Corleone crime f...,1974-12-20,en,Crime Drama
3,424,"[18, 36, 10752]",Schindler's List,The true story of how businessman Oskar Schind...,1993-12-15,en,Drama History War
4,19404,"[35, 18, 10749]",Dilwale Dulhania Le Jayenge,"Raj is a rich, carefree, happy-go-lucky second...",1995-10-19,hi,Comedy Drama Romance


Criando a tag unindo as variaveis

In [8]:
df_filmes['tags'] = df_filmes['resumo'] + ' ' + df_filmes['genero'] + ' ' + df_filmes['idioma_original'] + ' ' + df_filmes['lancamento']
df_filmes['tags']

0       Spanning the years 1945 to 1955, a chronicle o...
1       Framed in the 1940s for the double murder of h...
2       In the continuing saga of the Corleone crime f...
3       The true story of how businessman Oskar Schind...
4       Raj is a rich, carefree, happy-go-lucky second...
                              ...                        
9995    Ex-Federale agent Machete is recruited by the ...
9996    Deep in the African jungle, a baby named Georg...
9997    In the aftermath of a nuclear disaster, a star...
9998    Vitoria-Gasteiz, Basque Country, Spain, 2019. ...
9999    A group of animals waiting for the annual floo...
Name: tags, Length: 10000, dtype: object

In [9]:
df_filmes_process = df_filmes[['id', 'titulo', 'tags']]
df_filmes_process = df_filmes_process.drop_duplicates()
df_filmes_process.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   id      10000 non-null  int64 
 1   titulo  10000 non-null  object
 2   tags    9998 non-null   object
dtypes: int64(1), object(2)
memory usage: 234.5+ KB


## Vetorizando os dados

In [13]:
cv = CountVectorizer(max_features=10000, stop_words='english')    
vector = cv.fit_transform(df_filmes_process['tags'].values.astype('U')).toarray()
vector

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]], dtype=int64)

## Sistema de recomendação

In [14]:
similarity = cosine_similarity(vector)
similarity

d:\python\Sistema de recomendacao\env\lib\site-packages\sklearn\utils\extmath.py:189: RuntimeWarning: invalid value encountered in matmul
  ret = a @ b


array([[1.        , 0.13837968, 0.46714184, ..., 0.08421519, 0.02860648,
        0.05239625],
       [0.13837968, 1.        , 0.12659242, ..., 0.06085806, 0.03100868,
        0.        ],
       [0.46714184, 0.12659242, 1.        , ..., 0.09245003, 0.03140371,
        0.05751973],
       ...,
       [0.08421519, 0.06085806, 0.09245003, ..., 1.        , 0.1132277 ,
        0.10369517],
       [0.02860648, 0.03100868, 0.03140371, ..., 0.1132277 , 1.        ,
        0.0352235 ],
       [0.05239625, 0.        , 0.05751973, ..., 0.10369517, 0.0352235 ,
        1.        ]])

In [19]:
def recomendation_system(movie):
    id_of_movie = df_filmes_process[df_filmes_process['titulo']==movie].index[0]
    distances = similarity[id_of_movie]
    movie_list = sorted(list(enumerate(distances)), reverse=True, key=lambda x:x[1])[1:6]
    for movie_id in movie_list:
        print(df_filmes_process.iloc[movie_id[0]].titulo)

In [21]:
recomendation_system('Iron Man')

Iron Man 3
Guardians of the Galaxy Vol. 2
Avengers: Age of Ultron
Iron Man 2
Deathstroke: Knights & Dragons - The Movie
